In [29]:
from elasticsearch import Elasticsearch
from elasticsearch_llm_cache import ElasticsearchLLMCache, ElasticsearchLLMFilter
from elasticsearch.exceptions import NotFoundError

# common libraries
from dotenv import load_dotenv
import os
from os import environ
import openai
from icecream import ic

# load our environment file
load_dotenv()

es_url = f"https://{os.environ['elasticsearch_user']}:{os.environ['elasticsearch_pw']}@{os.environ['elasticsearch_host']}:{os.environ['elasticsearch_port']}"
# es_index= os.environ['elasticsearch_index']
os.environ['OPENAI_API_KEY'] = os.environ['openai_api_key']
open_api_key=os.environ['OPENAI_API_KEY']

# define our API Key
openai.api_key = os.getenv("openai_api_key")


In [30]:
es = Elasticsearch([es_url])

cache_index_name = 'llm_cache_test'
filter_index_name = 'llm_filter_test'
model_id = 'sentence-transformers__msmarco-minilm-l-12-v3'

# if es.ml.get_trained_models(model_id=model_id):
#     ic(f'{model_id} exists.')
# else:
#     ic(f'{model_id} does not exist.')

# if es.indices.exists(index=cache_index_name):
#     ic(f'{cache_index_name} exists.')
# else:
#     ic(f'{cache_index_name} does not exist.')
#     es.indices.create(index=cache_index_name)

# if es.indices.exists(index=filter_index_name):
#     ic(f'{filter_index_name} exists.')
# else:
#     ic(f'{filter_index_name} does not exist.')
#     es.indices.create(index=filter_index_name)

In [5]:
import os
import time
#print(os.environ['ELASTIC_CLOUD_ID'])
#time.sleep(10)
from elasticsearch import Elasticsearch

from elasticsearch_llm_cache import (
    ElasticsearchLLMCache,  
    ElasticsearchLLMFilter,
)

from pprint import pprint
import time

es_client = es

if es_client.indices.exists(index=cache_index_name):
    es_client.indices.delete(index=cache_index_name)
    ic(f'{cache_index_name} exists, deleting.')

if es_client.indices.exists(index=filter_index_name):
    es_client.indices.delete(index=filter_index_name)
    ic(f'{filter_index_name} exists, deleting.')

# es_client.indices.create(index=cache_index_name)
# es_client.indices.create(index=filter_index_name)


# Initialize your caching class
cache = ElasticsearchLLMCache(es_client=es_client, index_name=cache_index_name, es_model_id=model_id, create_index=False)
cache.create_index(dims=384)

filter = ElasticsearchLLMFilter(es_client=es_client, index_name=filter_index_name, es_model_id=model_id, create_index=False)
filter.create_index(dims=384)


INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.644s]
INFO:elastic_transport.transport:DELETE https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.101s]
ic| f'{cache_index_name} exists, deleting.': 'llm_cache_test exists, deleting.'
INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test [status:200 duration:0.034s]
INFO:elastic_transport.transport:DELETE https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test [status:200 duration:0.096s]
ic| f'{filter_index_name} exists, deleting.': 'llm_filter_test exists, deleting.'
INFO:elastic_transport.transport:HEAD https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:404 duration:0.032s]
INFO:elastic_transport.transport:PUT https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_cache_test [status:200 duration:0.151s]
ic| f

{'cache_index': 'llm_filter_test', 'created_new': True}

In [7]:
# filter.clear()
subjects = ['sports', 'money', 'celebrities', 'basketball', 'soccer', 'tennis', 'skiing', 'animals', 'snakes', 'software development']
filter_contents = []

for subject in subjects:
    filter.add(prompt=subject, response=f"Sorry, I can't talk about {subject}.")

# for subject in subjects:
#     filter_contents.append({'prompt': subject, 'response': f"Sorry, I can't talk about {subject}."})


# filter_results = filter.add_bulk(documents=filter_contents)
# filter.list()


INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.493s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_search [status:200 duration:0.036s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.035s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_doc [status:400 duration:0.036s]
ERROR:elasticsearch_llm_cache:BadRequestError(400, 'document_parsing_exception', '[1:7896] failed to parse: The [dot_product] similarity can only be used with unit-length vectors. Preview of invalid vector: [0.5523583, 0.0223902, 0.42353424, -0.35227466, -0.3553125, ...]')
INFO:elastic_transport.transport:POST https:/

In [8]:
results = filter.query(prompt_text="How do I code a linked list with python?", similarity_threshold=0.2, size=1)
ic(results)


ic| prompt_text: 'How do I code a linked list with python?'
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.037s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_search [status:200 duration:0.035s]
ic| results: False


False

In [10]:
filter.query('I will vote for a democratic politician', similarity_threshold=0.25)
ic(results)

ic| prompt_text: 'I will vote for a democratic politician'
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/sentence-transformers__msmarco-minilm-l-12-v3/_infer [status:200 duration:0.036s]
INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/llm_filter_test/_search [status:200 duration:0.033s]
ic| results: False


False

In [ ]:
prompt = "Hello, my name is Morgan?"
llm_response = "I'm here to assist you!"  # Assume this response is fetched from LLM

cache_contents = [
    {"prompt": "Hello, my name is Morgan", "response": "Hi Morgan"},
    {"prompt": "Hello, my name is Adam", "response": "Hi Adam"},
    {"prompt": "Hello, my name is David", "response": "Hi David"},
    {"prompt": "Hello, my name is Morgan", "response": "Hi Morgan"},
]

cache_results = cache.add_bulk(documents=cache_contents)

for item in cache_contents:
    llm_response = cache.query(prompt_text=item['prompt'], )
    ic(llm_response)



In [ ]:
!eland_import_hub_model --url "$es_url" \
      --hub-model-id "sentence-transformers/msmarco-MiniLM-L-12-v3" \
      --task-type "text_embedding"

In [20]:
!eland_import_hub_model --url "$es_url" \
      --hub-model-id "valhalla/distilbart-mnli-12-6" \
      --task-type "zero_shot_classification"

2023-11-17 12:57:06,650 INFO : Establishing connection to Elasticsearch
2023-11-17 12:57:06,972 INFO : Connected to cluster named '4dadf200942c4f3fb6113618e49a559c' (version: 8.11.0)
2023-11-17 12:57:06,976 INFO : Loading HuggingFace transformer tokenizer and model 'valhalla/distilbart-mnli-12-6'
/Users/morgan/Library/CloudStorage/Dropbox/me/prompt-eng/.venv/lib/python3.10/site-packages/transformers/models/bart/modeling_bart.py:239: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/Users/morgan/Library/CloudStorage/Dropbox/me/prompt-eng/.venv/lib/python3.10/site-packages/transformers/models/bart/modeling_bart.py:246: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorr

In [17]:
!eland_import_hub_model --url "$es_url" \
      --hub-model-id "distilbert-base-uncased-finetuned-sst-2-english" \
      --task-type "text_classification" \
      --start

2023-11-17 12:30:43,902 INFO : Establishing connection to Elasticsearch
2023-11-17 12:30:44,280 INFO : Connected to cluster named '4dadf200942c4f3fb6113618e49a559c' (version: 8.11.0)
2023-11-17 12:30:44,281 INFO : Loading HuggingFace transformer tokenizer and model 'distilbert-base-uncased-finetuned-sst-2-english'
/Users/morgan/Library/CloudStorage/Dropbox/me/prompt-eng/.venv/lib/python3.10/site-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)
2023-11-17 12:31:17,624 INFO : Creating model with id 'distilbert-base-uncased-finetuned-sst-2-english'
2023-11-17 12:31:17,685 INFO : Uploading model definition
100%|███████████

In [18]:
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

models = MlClient.get_trained_models(es_client)
for model in models["trained_model_configs"]:
    print(model["model_id"])


model_id = "distilbert-base-uncased-finetuned-sst-2-english"

#Run a query againt the model - this is the format the query imput must be used in, you can later map your features into this format through an ingest pipeline
doc_test = {"text_field": "This is a very nice sentence"}

result = MlClient.infer_trained_model(es, model_id =model_id, docs = doc_test)
result["inference_results"]


INFO:elastic_transport.transport:GET https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models [status:200 duration:0.354s]


.elser_model_1
.elser_model_2
distilbert-base-uncased-finetuned-sst-2-english
elastic__distilbert-base-uncased-finetuned-conll03-english
lang_ident_model_1
sentence-transformers__all-distilroberta-v1
sentence-transformers__msmarco-minilm-l-12-v3
valhalla__distilbart-mnli-12-6


INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/distilbert-base-uncased-finetuned-sst-2-english/_infer [status:200 duration:0.813s]


[{'predicted_value': 'POSITIVE', 'prediction_probability': 0.9997331448853626}]

In [31]:
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

def filter(text, labels, es, model_id = "valhalla__distilbart-mnli-12-6", threshold = 0.9):

  doc_test = {"text_field": text}
  inference_config = {
      "zero_shot_classification": {
        "labels": labels,
        "multi_label": True
      }
    }

  result = MlClient.infer_trained_model(es, model_id =model_id, docs = doc_test, inference_config=inference_config)

  filtered_results = [sub_item for item in result for sub_item in item['top_classes'] if sub_item['class_score'] > threshold]

  return filtered_results

res = filter(es=es, text="Our city councilman is going to be at the event.", labels=["sports", "money", "family", "politics"])
res

INFO:elastic_transport.transport:POST https://demo-defc18.es.us-central1.gcp.cloud.es.io:9243/_ml/trained_models/valhalla__distilbart-mnli-12-6/_infer [status:200 duration:0.375s]


TypeError: string indices must be integers

[{'class_name': 'politics',
  'class_probability': 0.988533928117081,
  'class_score': 0.988533928117081}]